# Loading Data into MySQL

The goal of this notebook is to show you how to load `unstructured` outputs into MySQL. This allows you to retrieve pre-processed text based on metadata fields that `unstructured` extracts.

If you don't have MySQL installed on your system yet, you can follow the instructions [here](https://dev.mysql.com/doc/refman/5.7/en/installing.html) to get it installed. If you haven't already, run `pip install -r requirements.txt` in the base directory of the example folder to install the Python dependencies.

# Preprocess Documents with Unstructured

First, we'll pre-process a few documents using the the `unstructured` libraries. The example documents are available under the `example-docs` directory in the `unstructured` repo. At the end of this section, we'll wind up with a list of `Element` objects that we can pass into an `unstructured` staging function.

In [1]:
import os

from unstructured.partition.auto import partition

In [2]:
# NOTE: Update this directory if you are running the notebook
# from somewhere other than the examples/mysql folder in the
# unstructured repo
EXAMPLE_DOCS_FOLDER = "../../example-docs/"

In [3]:
documents_to_process = [
    "fake-email.eml",
    "fake.docx",
    "layout-parser-paper-fast.pdf",
]

In [4]:
elements = []
for document in documents_to_process:
    filename = os.path.join(EXAMPLE_DOCS_FOLDER, document)
    elements.extend(partition(filename=filename, strategy="fast"))

In [5]:
elements[0].text

'This is a test email to use for unit tests.'

In [6]:
elements[0].metadata.to_dict()

{'filename': '../../example-docs/fake-email.eml',
 'date': '2022-12-16T17:04:16-05:00',
 'sent_from': ['Matthew Robinson <mrobinson@unstructured.io>'],
 'sent_to': ['Matthew Robinson <mrobinson@unstructured.io>'],
 'subject': 'Test Email'}

## Convert the Unstructured Outputs to a Dataframe

Now that we have the document outputs as a list of `Element` objects, we can convert the list to a dataframe using the `convert_to_dataframe` staging function. With the elements in dataframe format, we can now see the text and type along side various document metadata.

In [7]:
from unstructured.staging.base import convert_to_dataframe

In [8]:
elements_df = convert_to_dataframe(elements)

In [9]:
elements_df.head()

,type,text,element_id,coordinates,filename,page_number,url
0,NarrativeText,This is a test email to use for unit tests.,f49fbd614ddf5b72e06f59e554e6ae2b,NaN,../../example-docs/fake-email.eml,NaN,NaN
1,Title,Important points:,9c218520320f238595f1fde74bdd137d,NaN,../../example-docs/fake-email.eml,NaN,NaN
2,ListItem,Roses are red,8522061b991b1db70453502d328fe07e,NaN,../../example-docs/fake-email.eml,NaN,NaN
3,ListItem,Violets are blue,c3c4527761d4e4b8d0a4c4a0d46954c8,NaN,../../example-docs/fake-email.eml,NaN,NaN
4,Title,Lorem ipsum dolor sit amet.,dd14cbbf0e74909aac7f248a85d190af,NaN,../../example-docs/fake.docx,NaN,NaN


## Load the Documents into MySQL

Once the `unstructured` elements are converted to a dataframe, we can easily upload them to MySQL using built-in `pandas` utilities. In this case, we'll upload the documents using a connection created with the `sqlalchemy` libary. 

Run `export MYSQL_PWD=<my-password>` to store your MySQL password in as an environment variable. You can accomplish this using other MySQL clients as well. In the `elements_df.to_sql` block, you can change `if_exists` to `"append"` if you would like to add to a table instead of replacing it.

In [10]:
import os

import pandas as pd
from sqlalchemy import create_engine, text

In [11]:
# NOTE: update these values to reflect the username/password/database
# name that you created in MySQL
user = "matt"
pwd = os.environ.get("MYSQL_PWD")
host = "localhost"
db = "unstructured_example"

In [12]:
engine = create_engine(
    f"mysql+mysqlconnector://{user}:{pwd}@{host}/{db}",
)

In [13]:
table_name = "processed_documents"

In [14]:
elements_df.to_sql(name=table_name, con=engine, if_exists="replace", index=False)

-1

## Read the Documents from MySQL

Now that the documents are loaded into MySQL, you can run queries that retrieve document snippets based on metadata that `unstructured` has extracted. In this case, we show an example of how to retrieve all of the narrative text from a specific document.

In [15]:
sql = """
SELECT *
FROM unstructured_example.processed_documents
WHERE type = "NarrativeText"
AND filename LIKE '%fake-email.eml%'
"""

In [16]:
with engine.begin() as conn:
    elements_read_df = pd.read_sql_query(sql=text(sql), con=conn)

In [17]:
elements_read_df.head()

,type,text,element_id,coordinates,filename,page_number,url
0,NarrativeText,This is a test email to use for unit tests.,f49fbd614ddf5b72e06f59e554e6ae2b,None,../../example-docs/fake-email.eml,None,None
